In [17]:
from pyspark.sql import SparkSession

def get_spark_session(app_name: str = "LinkITAssignment"):
    """
    Creates and returns a SparkSession.
    """
    builder = (
        SparkSession.builder
        .appName(app_name)
    )
    
    return builder.getOrCreate()


In [10]:
from faker import Faker
import random
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

def generate_customer_data(spark, num_records=200, seed=42):
    Faker.seed(seed)
    random.seed(seed)
    
    data = []
    for _ in range(num_records):
        customer_number = f"{fake.lexify(text='????')}{fake.numerify(text='####')}"
        record = (
            customer_number,
            fake.first_name(),
            fake.last_name(),
            fake.date_of_birth(minimum_age=18).strftime("%Y-%m-%d"),
            fake.ssn(),
            fake.street_name(),
            random.randint(1, 9999),
            fake.city(),
            fake.state(),
            fake.country(),
            fake.postcode(),
            fake.credit_card_number(),
            fake.credit_card_expire(),
            fake.credit_card_security_code(),
            fake.credit_card_provider()
        )
        data.append(record)

    schema = StructType([
        StructField("CUSTOMER_NUMBER", StringType(), False),
        StructField("FIRST_NAME", StringType(), False),
        StructField("LAST_NAME", StringType(), False),
        StructField("BIRTH_DATE", StringType(), False),
        StructField("SSN", StringType(), False),
        StructField("CUSTOMER_ADDRESS_STREET", StringType(), False),
        StructField("CUSTOMER_ADDRESS_HOUSE_NUMBER", IntegerType(), False),
        StructField("CUSTOMER_ADDRESS_CITY", StringType(), False),
        StructField("CUSTOMER_ADDRESS_STATE", StringType(), False),
        StructField("CUSTOMER_ADDRESS_COUNTRY", StringType(), False),
        StructField("CUSTOMER_ADDRESS_ZIP_CODE", StringType(), False),
        StructField("CREDITCARD_NUMBER", StringType(), False),
        StructField("CREDITCARD_EXPIRATION_DATE", StringType(), False),
        StructField("CREDITCARD_VERIFICATION_CODE", StringType(), False),
        StructField("CREDITCARD_PROVIDER", StringType(), False)
    ])

    return spark.createDataFrame(data, schema)


In [11]:
from faker import Faker
import random
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

fake = Faker()

def generate_product_transactions(spark, txn_ids):
    departments = [fake.word() for _ in range(50)] 
    
    product_data = []
    for txn_id in txn_ids:
        num_items = random.randint(1, 5)
        for _ in range(num_items):
            product_data.append((
                txn_id,
                fake.ean13(),
                random.choice(departments),
                round(random.uniform(1, 100000), 2),
                random.randint(1, 200)
            ))

    schema = StructType([
        StructField("TRANSACTION_ID", StringType(), False),
        StructField("ITEM_EAN", StringType(), False),
        StructField("ITEM_DEPARTMENT", StringType(), False),
        StructField("ITEM_VALUE", DoubleType(), False),
        StructField("ITEM_QUANTITY", IntegerType(), False)
    ])

    return spark.createDataFrame(product_data, schema)


In [12]:
import uuid
import random
from faker import Faker
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

fake = Faker()

def generate_transaction_data(spark, df_customers, num_records=2000):
    customers = [row["CUSTOMER_NUMBER"] for row in df_customers.collect()]
    
    transactions_data = []
    for _ in range(num_records):
        transaction_id = str(uuid.uuid4())  
        customer_number = random.choice(customers)
        transaction_date_time = fake.date_time_between(
            start_date=datetime(2022, 1, 1), 
            end_date=datetime(2022, 12, 31)
        )   
        
        record = (
            transaction_id,
            customer_number,
            0.0,
            transaction_date_time
        )
        transactions_data.append(record)

    schema = StructType([
        StructField("TRANSACTION_ID", StringType(), False),
        StructField("CUSTOMER_NUMBER", StringType(), False),
        StructField("TRANSACTION_VALUE", DoubleType(), False),
        StructField("TRANSACTION_DATE_TIME", TimestampType(), False)
    ])
    
    return spark.createDataFrame(transactions_data, schema)


In [13]:
from pyspark.sql import functions as F

def update_transaction_values(df_transactions, df_products):
    txn_sums = (
        df_products
        .withColumn("TOTAL_ITEM_VALUE", F.col("ITEM_VALUE") * F.col("ITEM_QUANTITY"))
        .groupBy("TRANSACTION_ID")
        .agg(F.sum("TOTAL_ITEM_VALUE").alias("NEW_TRANSACTION_VALUE"))
    )

    df_transactions_updated = (
        df_transactions
        .join(txn_sums, on="TRANSACTION_ID", how="left")
        .withColumn(
            "TRANSACTION_VALUE", 
            F.coalesce(F.col("NEW_TRANSACTION_VALUE"), F.col("TRANSACTION_VALUE"))
        )
        .drop("NEW_TRANSACTION_VALUE")
    )

    # Add year and month (partitions) columns based on the TRANSACTION_DATE_TIME column
    df_transactions_updated = (
        df_transactions_updated
        .withColumn(
            "ANO_MES", 
            F.date_format(F.col("TRANSACTION_DATE_TIME"), "yyyy-MM"))
    )

    return df_transactions_updated

In [14]:
def top_customers(transactions_df):
    costumers_result = (transactions_df
              .select("CUSTOMER_NUMBER", "TRANSACTION_ID", "TRANSACTION_VALUE", "TRANSACTION_DATE_TIME")
              .orderBy(F.desc("TRANSACTION_VALUE"))
              .limit(100))
    
    return costumers_result


def total_by_provider(customers_df, transactions_df, products_df):
    joined_df = (transactions_df
                .join(customers_df, "CUSTOMER_NUMBER", "inner")
                .join(products_df, "TRANSACTION_ID", "inner"))
    
    # Filter transactions between 00:00 and 12:00
    filtered_df = joined_df.filter(
        (F.hour(F.col("TRANSACTION_DATE_TIME")) >= 0) & 
        (F.hour(F.col("TRANSACTION_DATE_TIME")) < 12)
    )

    total_result = (filtered_df
              .withColumn("TOTAL_ITEM_VALUE", F.col("ITEM_VALUE") * F.col("ITEM_QUANTITY"))
              .groupBy("CREDITCARD_PROVIDER")
              .agg(
                  F.sum("ITEM_QUANTITY").alias("TOTAL_QUANTITY"),
                  F.sum("TOTAL_ITEM_VALUE").alias("TOTAL_VALUE")
              )
             )
    
    return total_result

In [18]:
from pyspark.sql import SparkSession

try:
    spark
except NameError:
    spark = SparkSession.builder \
        .appName("MeuApp") \
        .getOrCreate()

customer_df = generate_customer_data(spark)
#customer_df.write.mode("overwrite").parquet("data/bronze/customers")

creditcard_df = generate_transaction_data(spark, customer_df)
#creditcard_df.write.mode("overwrite").parquet("data/bronze/transactions")

txn_ids = [row["TRANSACTION_ID"] for row in creditcard_df.collect()]
df_products = generate_product_transactions(spark, txn_ids)
#df_products.write.mode("overwrite").parquet("data/bronze/products")


# === PHASE 2: DATA TRANSFORMATION ===
df_transactions_updated = update_transaction_values(creditcard_df, df_products)
df_transactions_updated.show(truncate=False)
#df_transactions_updated.write.mode("overwrite").partitionBy("ANO_MES").parquet("data/gold/totals_by_provider")



# === PHASE 3: ANALYTICS (GOLD) ===
print(">> Top 100 Customers:")
top_customers_df = top_customers(df_transactions_updated)
top_customers_df.show(truncate=False)
#top_customers_df.write.mode("overwrite").parquet("data/gold/top_customers")


+------------------------------------+---------------+--------------------+---------------------+-------+
|TRANSACTION_ID                      |CUSTOMER_NUMBER|TRANSACTION_VALUE   |TRANSACTION_DATE_TIME|ANO_MES|
+------------------------------------+---------------+--------------------+---------------------+-------+
|8f9a939d-60d1-42bf-9206-1db0210f8962|Drtq3135       |1.3222311469999999E7|2022-10-17 00:52:59  |2022-10|
|a8bd62c4-b73b-479a-854f-9e400741d0af|FulF3545       |3.830979895E7       |2022-06-07 07:43:38  |2022-06|
|adf91747-c447-490c-9b7e-fdc45a7db0d1|QdWa6231       |1.3882797079999998E7|2022-06-03 09:38:27  |2022-06|
|fdd232bc-cdf3-44d5-b8ee-a0143ab30928|ZaMF3397       |7515637.9399999995  |2022-09-19 16:18:01  |2022-09|
|0d7fbf67-63de-49ec-ad07-e74d0f5503cc|vPfC5183       |9404605.6           |2022-10-09 21:59:04  |2022-10|
|c1043f34-ebf8-4cf7-b543-3dac3cd78457|Gsvh8489       |9978404.309999999   |2022-09-20 04:24:33  |2022-09|
|f64cdb36-d73f-45de-81d1-c526f7157e3b|WaHn3428

In [19]:
df_transactions_updated.count()

2000